# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")

using SimilaritySearch, TextSearch, Plots, KNearestCenters, LinearAlgebra, Embeddings, HypertextLiteral, CodecZlib, JSON, Base64, Random
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
include("knn.jl")

scores (generic function with 1 method)

In [3]:
struct ValidVocabulary{DictType} <: AbstractTokenTransformation
    valid::DictType
end

TextSearch.transform_unigram(S::ValidVocabulary, tok) = haskey(S.valid, tok) ? tok : nothing

function text_model_and_vectors(
        corpus, vocab;
        localweighting=TpWeighting(),
        globalweighting=BinaryGlobalWeighting(),
        nlist=[1],
        qlist=[],
        slist=[],
        group_usr=true,
        group_url=true,
        group_num=true,
        del_diac=true,
        lc=true
    )

    voc = Vocabulary(length(corpus))
    for v in vocab
       push!(voc, v)
    end
    
    tt = ValidVocabulary(voc.token2id)
    textconfig = TextConfig(; group_usr, group_url, del_diac, lc, group_num, nlist, qlist, slist, tt)
    tokenize_and_append!(voc, textconfig, corpus)
    model = VectorModel(globalweighting, localweighting, voc)
    (; textconfig, model, voc)
end


text_model_and_vectors (generic function with 1 method)

In [4]:
function embeddings(embname="MX.vec")
    embfile = "../data/$embname"
    !isfile(embfile) && download("http://geo.ingeotec.mx/~sadit/regional-spanish-models/$embname", embfile)
    emb = load_embeddings(FastText_Text, embfile)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end
    
    (; X=emb.embeddings, vocab=emb.vocab)
end

function vectorize_as_prototype!(c, text, E, T)
    x = vectorize(T.model, T.textconfig, text; normalize=false)
    if length(x) == 1 && haskey(x, 0)
        #@warn "empty vector $(Int(i)) selecting a random vector for it " # $(corpus[i])
        rand!(c)
    else
        for (id, weight) in x
            id == 0 && continue
             c .= c .+ weight .* view(E.X, :, id)
        end
    end
    
    normalize!(c)
end

function vectorize_corpus_as_prototypes(corpus, E, T)
    dim = size(E.X, 1)
    n = length(corpus)
    C = zeros(Float32, dim, n)
    # Threads.@threads
    for i in 1:n
        vectorize_as_prototype!(view(C, :, i), corpus[i], E, T)
    end
    
    C
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 1 method)

In [5]:
@time E = embeddings()

 34.921058 seconds (11.18 M allocations: 11.654 GiB, 2.47% gc time, 1.37% compilation time)


(X = Float32[-0.039261177 -0.044748805 … -0.051247735 0.05427054; 0.09036921 0.03974378 … -0.017566958 0.06556069; … ; -0.043778244 0.04899757 … -0.038878556 -0.097423464; 0.021343287 0.012799488 … -0.0017221138 0.006681864], vocab = ["</s>", "_usr", "que", "de", ",", ".", "y", "a", "la", "no"  …  "tannehil", "movelike", "peliblanca", "cristianzav", "rqq", "preguntartee", "sutedes", "venciendolo", "eviscerado", "fectivo"])

In [15]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_news()
T = text_model_and_vectors(D.corpus, E.vocab)
@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau", "AristeguiOnline", "el_pais", "EFEnoticias", "PublimetroMX", "PublimetroChile", "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"], {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=30244, maxoccs=43245})


(["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau"  …  "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"], {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=30244, maxoccs=43245})

In [16]:
@time C = vectorize_corpus_as_prototypes(D.corpus, E, T)

  1.753261 seconds (3.34 M allocations: 313.884 MiB, 31.21% compilation time)


300×30244 Matrix{Float32}:
 -0.052207    -0.0267733    -0.0534574    …  -0.0799066   -0.10502
  0.0333062   -0.00691014   -0.00203628       0.0161733    0.0393123
 -0.0489505    0.000651152  -0.00835875      -0.00500099   0.0214965
 -0.00438946   0.00205949   -0.00268551       0.0346591    0.0259731
  0.0589265    0.0193912     0.0118446       -0.0200484   -0.00529611
 -0.00172394  -0.0041373     0.03163      …   0.0155074    0.0160663
 -0.0612389   -0.00260272    0.00934668      -0.00196163  -0.00115164
  0.0852119    0.00278283   -0.00549386      -0.038946     0.0171894
  0.0507986    0.0329853     0.0873546        0.0999684    0.120996
  0.017137    -0.0118347     0.000876286     -0.0145413   -0.0141206
  0.0676252    0.0417673     0.0835937    …   0.0804474    0.0961554
  0.055106    -0.0198022     0.0331076        0.0237437    0.0534497
  0.0311947   -0.011102      0.0589837        0.0552063    0.0185002
  ⋮                                       ⋱               
 -0.0635303   -0.0

### Se crea el índice métrico

In [17]:
@time index = create_index(MatrixDatabase(C), 0.95)

  8.005981 seconds (13.06 M allocations: 1.497 GiB, 9.57% gc time, 85.28% compilation time)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  links: Array{Vector{Int32}}((30244,))
  locks: Array{Base.Threads.SpinLock}((30244,))
  hints: Array{Int32}((96,)) Int32[6, 52, 229, 238, 260, 295, 338, 352, 363, 464  …  2455, 2572, 2576, 2612, 2666, 2672, 2734, 2787, 2808, 25642]
  search_algo: BeamSearch
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [18]:
let k = 32
    Qvectors = MatrixDatabase(vectorize_corpus_as_prototypes(Q.corpus, E, T))
    t1 = @elapsed I, _ = searchbatch(index, Qvectors, k)
    # WARNING: Don't run the following line, it takes too much time
    ex = ExhaustiveSearch(; db=index.db, dist=index.dist)
    t2 = @elapsed gI, _ = searchbatch(ex, Qvectors, k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.labels)
    s1 = scores(Q.labels, knn(I, labels))
    s2 = scores(Q.labels, knn(gI, labels))
    
    @htl """
    <div>(brute vs indexed) macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

<div>(brute vs indexed) macro-recall: 0.9498940047604879, n: 3361</div>
<div>searchgraph search time: 0.733757938, scores: <span class="Core-NamedTuple">(macrof1 = 0.23000048969948245, macrorecall = 0.24310808517812765, accuracy = 0.2728354656352276)</span></div>
<div>brute force search time: 0.634220118, scores: <span class="Core-NamedTuple">(macrof1 = 0.24511644280600453, macrorecall = 0.2547021687929789, accuracy = 0.283844094019637)</span></div>

### Búsqueda y presentación de los resultados

In [19]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [20]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.000411 seconds (2 allocations: 320 bytes)


1,5559,0.113,Radio_Formula,El presidente Andrés Manuel López Obrador exhibió el supuesto sueldo de Loret de Mola... con un documento con errores ortográficos. https://t.co/nx4bDFfw8p
2,3647,0.122,CNNEE,Así pensó un plan B el presidente de México Andrés Manuel López Obrador para organizar la consulta de revocación de mandato. https://t.co/GaAeHdjhAG
3,383,0.126,azucenau,Informa @RicardoAnayaC que ha sido citado en el Reclusorio Norte por lo que acusa persecución de parte del presidente Andrés Manuel López Obrador
4,27877,0.127,azucenau,"#ALMOMENTO | Por segundo día consecutivo, el presidente Andrés Manuel López Obrador salió esta tarde de Palacio Nacional. https://t.co/tdBqBYhhuc"
5,2292,0.129,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate El gobernador Ricardo Gallardo Cardona pidió al presidente Andrés Manuel López Obrador apoyo para reforzar l…
6,24904,0.135,azucenau,"#AzucenaxMilenio | Durante #LaMañanera, el presidente López Obrador respaldó nuevamente al gobernador de Veracruz, Cuitláhuac García https://t.co/iX2ubBHbyJ"
7,1231,0.136,azucenau,"#ENVIVO | Mensaje conjunto del presidente López Obrador y Alejandro Giammattei, presidente de #Guatemala https://t.co/uOwcCHGKt3"


  0.001270 seconds (2 allocations: 1.438 KiB)


1,22544,0.245,LaRazon_mx,#SSPC: Red de tráfico de migrantes es ubicada en #Guanajuato y #Tamaulipas https://t.co/2TdiV3G2Zf https://t.co/hctIUWIx7G
2,22011,0.263,AristeguiOnline,El comercio ilegal de drogas se digitaliza con la pandemia https://t.co/dzZtvcjBAA https://t.co/hrG2x5a5xs
3,5839,0.284,AristeguiOnline,Dos presuntos asesinos de taxistas de la Ciudad de México fueron detenidos https://t.co/I1WMMoLgzO
4,3848,0.285,Reforma,"El 6 de enero de 2021 inició el juicio de Joaquín Naasón García Los cargos estaban relacionados con el abuso de menores, producción de pronografía e incluso tráfico de personas, entre otros. https://t.co/8qwEvydrf4"
5,9124,0.285,EFEnoticias,"Un centenar de profesores mantienen cortado esta mañana el tráfico en la Ronda Litoral, a la altura de la Vila Olímpica de Barcelona, lo que está causando grandes retenciones de tráfico en esta vía en el tercer día de huelga de la enseñanza. https://t.co/jIsziPxkn7"
6,16142,0.286,Reforma,#Entérate Intentos de asalto y poca infraestructura carretera en el Estado de México obstaculiza el reparto de ayuda del Banco de Alimentos Poniente https://t.co/MPpkMmpRX8
7,23285,0.287,El_Universal_Mx,"De acuerdo con datos de la Administración de Control de Drogas (DEA) de Estados Unidos, en la versión original, el fentanilo es considerado de 30 a 50 veces más potente que la heroína https://t.co/9mWdq9W8PA https://t.co/bqchRWkYvZ"


  0.001338 seconds (6 allocations: 7.375 KiB)


1,28803,0.452,bbcmundo,Los vacunados transmiten menos el virus. https://t.co/BYtDcmYqwj
2,7976,0.462,abc_es,Primeros datos de la eficacia de dos antivirales frente al virus de la viruela del simio https://t.co/TdBHvIIT3w
3,3632,0.468,AristeguiOnline,Molnupiravir elimina el virus SARS-CoV-2 activamente infeccioso al tercer día de comenzar la terapia: estudio https://t.co/93yBkWlj3G
4,6122,0.476,El_Universal_Mx,Es el primer deceso registrado en la entidad por esa variante del virus SARS-CoV-2 https://t.co/3FlnzSNpbk
5,15380,0.477,bbcmundo,El alentador efecto de la vacuna contra el virus del papiloma humano (VPH) en la reducción del cáncer de cuello uterino https://t.co/rQV9L9Q9F4
6,14629,0.482,SinEmbargoMX,Fármacos contra enfermedad inflamatoria intestinal protegerían de la COVID grave https://t.co/b7yXzIGahI https://t.co/nGZVBpX9Xi
7,9472,0.484,TelemundoNews,⚠️ 💊 @Pfizer retira del mercado algunos medicamentos contra la hipertensión por posible riesgo de cáncer. Otros medicamentos genéricos también fueron sacados del mercado. https://t.co/JJ3oac4Gek


In [21]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.000538 seconds (4 allocations: 1.750 KiB)


1,3316,0.0,julioastillero,RT @DeTodos_Modos: 🗞️😎 @julioastillero es un periodista que mezcla el oficio periodístico con la pasión por la política 👊🏾 ¡Esta noche @Jo…
2,21659,0.081,julioastillero,RT @adituzita: 🙌🏽✨ No se pierdan esta entrevista que le haré a @etiennista en el programa de @julioastillero. La violencia y la intoleranci…
3,13193,0.082,teleSURtv,RT @EnMediatica: Hoy @EnMediática todo el análisis de la agenda noticiosa con @OrlandoPerezEC sólo por @teleSURtv. Empezamos..! https://t.c…
4,27678,0.082,teleSURtv,RT @EnMediatica: Hoy @EnMediática todo el análisis de la agenda noticiosa con @OrlandoPerezEC sólo por @teleSURtv. Empezamos..! https://t.…
5,17046,0.082,teleSURtv,RT @EnMediatica: Hoy @EnMediática todo el análisis de la agenda noticiosa con @OrlandoPerezEC sólo por @teleSURtv. Empezamos..! https://t.c…
6,20716,0.082,teleSURtv,RT @EnMediatica: Hoy @EnMediática todo el análisis de la agenda noticiosa con @OrlandoPerezEC sólo por @teleSURtv. Empezamos..! https://t.…
7,21510,0.089,NTN24,RT @LaNocheNTN24: Es tema en @LaNocheNTN24 | No se pierda la gran entrevista de @CGurisattiNTN24 con el candidato presidencial por el @Coal…


  0.000709 seconds (4 allocations: 1.750 KiB)


1,19425,0.0,abrahamendieta,"No solo se buscaba proteger a Margarita Zavala, también a la familia Gómez del Campo, que no se nos olvide."
2,5650,0.062,abrahamendieta,"@Mabelle2606 @AguedaBc73 @DiputadosMorena @AndreaChavezTre @ChicShion La presentó el Senado, las modificaciones son el mecanismo parlamentario que te permite devolverlo. Y no está mal que se presente y se hable de ese tema, pero no así, que lo cambien y se vuelve a discutir, pero que no se abandone, porque hay que proteger a los trabajadores."
3,28668,0.071,SinEmbargoMX,"🚨 #URGENTE | José Ramón Cossío dijo que no sabe de dónde AMLO le agarró tirria, y que a lo mejor hay alguien que lo mal aconseja, pues conoce a los X. González, pero no se relaciona con ellos https://t.co/DNvQu1wpg3 https://t.co/IztRPMlghZ"
4,4361,0.074,el_pais,"🔊 El segundo día se lo dedicamos a Feijóo, candidato a las forzadas primarias del PP. Nos preguntamos quién es y cuál es su plan con dos de los periodistas que más le conocen. Porque, pese a sus años en la política, sigue siendo desconocido para muchos https://t.co/QRQ8SweTOQ https://t.co/KB0NUfatWV"
5,5877,0.074,abrahamendieta,"Se indigna Margarita Zavala cada vez en la Cámara de Diputados le recuerdan Guardería ABC. Hoy, Zaldivar cuenta como Calderón lo presionó con todo para intentar proteger a la familia Zavala. Prohibido olvidar."
6,20047,0.074,PublimetroChile,"En medio de la discusión por la crisis migratoria, Felipe Kast le dijo a Navarro que no tenía derecho a nada, porque no defendía los DDHH. https://t.co/qECQ6wKGHx"
7,12275,0.075,AdriDelgadoRuiz,"""El problema es que la gente se conduce como si ya no hubiera pandemia. Basta salir un minuto a la calle para darse cuenta que se saturan lugares y que no se usa el cubreboca"". Dijo el Dr. Antonio Lazcano para #ElDedoenlaLLaga. #ElHeraldoRadio del @heraldodemexico."


  0.000568 seconds (4 allocations: 1.750 KiB)


1,19989,0.0,el_pais,En España el Gobierno y la mayoría de comunidades han decidido esperar. ¿Significa esto que estamos abocados a un desbordamiento de los hospitales? No. Lo que significa es que estamos apostando con más optimismo o menos cautela https://t.co/qMgQVm6TEE
2,13065,0.046,el_pais,"""La covid cambia las cosas, es difícil que los votantes aprueben recortes en sanidad después de lo que ha pasado. Y creo que hay algo nuevo flotando en el aire: se pueden ganar elecciones diciendo que hay que gravar más a los ultrarricos"" https://t.co/JL47P4HAqH"
3,25178,0.048,TelemundoNews,"🗣 El presidente de #México niega que su país esté dominado por la violencia: ""No es lo que dicen"". Las balaceras y los asesinatos van en aumento, por eso el Gobierno de EE.UU. amplió una alerta de viaje a este país para sus ciudadanos. https://t.co/vMNQu3DJ5N"
4,23174,0.048,epigmenioibarra,"Lo que enloquece a la derecha, empeñada en negar el peso histórico del 2018, es la certeza de que si AMLO tiene un testamento político habremos millones de mexicanas y mexicanos decididos -en el caso de que fuera necesario- a garantizar que la voluntad en él expresada se cumpla."
5,5290,0.049,CNNEE,"Confirman en Nueva York cinco casos de la variante ómicron del coronavirus. No todos son de viajeros, lo que implica que ya hay transmisión comunitaria. Esto es lo que sabemos https://t.co/A3laqbBbik https://t.co/A3WEpXJt9f"
6,29240,0.049,bbcmundo,"""Muchas personas no quieren pagar impuestos. Eso es un sentimiento que creo que es común en muchos de los países de la región centroamericana y latinoamericana, porque al final las personas no ven los beneficios"" https://t.co/zmIkn2jlcf"
7,24553,0.05,AdriDelgadoRuiz,"¿Los restos de la #Malinche están enterrados en #Jilotepec? “Hay versiones que indican que sí, tenemos una de las iglesias más antiguas del país y tenemos tanta historia y riqueza cultural que es posible pensar que sí”: Rodolfo Noguez presidente Municipal de #Jilotepec https://t.co/ojzBF5lwid"


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
